# COVID-19 Dashboard

## Imports

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from bokeh.io import output_file, output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, GeoJSONDataSource, CDSView, GroupFilter
from bokeh.models import Panel, Tabs, HoverTool, LinearColorMapper, ColorBar
from bokeh.palettes import brewer, Dark2
from bokeh.tile_providers import get_provider, Vendors

## Settings

In [ ]:
output_notebook()

## Get Data

World data from [The Center for Systems Science and Engineering (CSSE) at JHU](https://systems.jhu.edu) using their [COVID-19](https://github.com/CSSEGISandData/COVID-19) repository on [Github](https://github.com/).

In [ ]:
def get_data_csse(cases=['confirmed', 'deaths', 'recovered']):
    """
    Return DataFrame with international data
    """
    
    base_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
    
    datasets = dict()
    
    for case in cases:
        url = base_url + f'csse_covid_19_time_series/time_series_covid19_{case}_global.csv'
        df = pd.read_csv(url)
        df = df.melt(['Province/State', 'Country/Region', 'Lat', 'Long'],
                     var_name='Date', value_name='Count')
        df['Date'] = pd.to_datetime(df['Date'])
        df['Case'] = case.capitalize()
    
        datasets[case] = df
        
    # Merge datasets
    df = pd.concat(datasets.values(), axis=0).reset_index(drop=True)
    
    return df

In [ ]:
df = get_data_csse(cases=['confirmed', 'deaths', 'recovered'])
df.sample(n=3)

In [ ]:
def wgs84_to_web_mercator(df,
                          lon_from='lon', lat_from='lat',
                          lon_to='longitude', lat_to='latitude'
                         ):
    """
    Convert decimal longitude/latitude to Web Mercator format.
    """
    k = 6378137
    df[lon_to] = df[lon_from] * (k * np.pi/180.0)
    df[lat_to] = np.log(np.tan((90 + df[lat_from]) * np.pi/360.0)) * k
    return df

In [ ]:
df = wgs84_to_web_mercator(df, lon_from='Long', lat_from='Lat').drop(['Lat', 'Long'], axis=1)
df.sample(n=3)

# Prepare plotting

In [ ]:
def normalize(values):
    min_value = min(values)
    max_value = max(values)    
    return [(x - min_value) / (max_value - min_value) for x in values]

In [ ]:
def size(values, min_size, max_size):
    return [int(np.ceil((x * (max_size - min_size)) + min_size)) for x in values]

In [ ]:
# set plot size
min_size = 10
max_size = 200

In [ ]:
# create and populate plot_size column
df['plot_size'] = -99 # initialize
for case in df['Case'].unique():
    df.loc[df['Case']==case, 'plot_size'] =\
    size(normalize(df.loc[df['Case']==case, 'Count']), min_size, max_size)

In [ ]:
# most recent data
most_recent_data = df[df['Date']==df['Date'].max()]\
.fillna('-')\
.rename(columns={'Province/State': 'state', 'Country/Region': 'country'})

## Visualize data

In [ ]:
# world maps
tile_provider = get_provider(Vendors.STAMEN_TERRAIN_RETINA)

source = ColumnDataSource(most_recent_data)

plots = list()

for case, color in zip(['Confirmed', 'Recovered', 'Deaths'],
                       ['crimson', 'forestgreen', 'navy']):
    
    case_filter = GroupFilter(column_name='Case', group=case)
    case_view = CDSView(source=source, filters=[case_filter])

    tooltips = [('Country/Region', '@country'), ('Province/State', '@state'), ('Cases', '@Count')]

    p = figure(plot_width=900, plot_height=500,
               x_range=(-18_000_000, 18_000_000), y_range=(-5_000_000, 8_500_000),
               x_axis_type='mercator', y_axis_type='mercator',
               tooltips=tooltips)
    p.add_tile(tile_provider)
    p.circle(x='longitude', y='latitude', size='plot_size', color=color, alpha=0.6,
             source=source, view=case_view)
    
    plots.append(Panel(child=p, title=case))
    
world_maps = Tabs(tabs=plots)
show(world_maps)

## Prepare plotting

`Province/State` data grouped by `Country/Region`.

In [ ]:
df_country = df.groupby(by=['Country/Region', 'Date', 'Case'])['Count'].sum()\
.reset_index()\
.rename(columns={'Country/Region': 'Country'})
df_country['DateString'] = [d.strftime('%d-%m-%Y') for d in df_country['Date']] # add DateString for hovertool to use
df_country.sample(n=7)

## Visualize data

In [ ]:
n = 5 # number of countries

plots = list()

for case in ['Confirmed', 'Recovered', 'Deaths']:

    top_countries = df_country.loc[df_country['Case']==case, ['Country', 'Count']]\
    .groupby(by=['Country'])['Count'].max().sort_values(ascending=False).head(n).index
    
    p = figure(plot_width=900, plot_height=500, x_axis_type='datetime')
    p.title.text = 'Click on legend entries to hide the corresponding lines'
    
    for data, country, color\
    in zip([ColumnDataSource(df_country[(df_country['Country'] == country) & (df_country['Case'] == case)])\
            for country in top_countries],
           top_countries,
           Dark2[n]):
        
        p.line(x='Date', y='Count', line_width=2, color=color, alpha=0.8, legend_label=country, source=data)
        hover = HoverTool(tooltips = [('Country', '@Country'), ('Date', '@DateString'), ('Count','@Count')])
        p.add_tools(hover)
        
    p.legend.location = 'top_left'
    p.legend.click_policy = 'hide'
    
    plots.append(Panel(child=p, title=case))
    
top_five = Tabs(tabs=plots)
show(top_five)

## Visualize data

In [ ]:
n = 5 # number of countries

ref_country = 'Netherlands'

plots = list()

for case in ['Confirmed', 'Recovered', 'Deaths']:
    
    ref_group = df_country.loc[df_country['Case']==case, ['Country', 'Count']]\
    .groupby(by=['Country'])['Count'].max().sort_values(ascending=False)\
    .to_frame().reset_index()
    id_nl = ref_group[ref_group['Country']==ref_country].index[0]
    peer_countries = ref_group.iloc[id_nl-int(np.ceil((n-1)/2)):id_nl+int(np.ceil((n-1)/2)+1), 0]
    
    p = figure(plot_width=900, plot_height=500, x_axis_type='datetime')
    p.title.text = 'Click on legend entries to hide the corresponding lines'
    
    for data, country, color\
    in zip([ColumnDataSource(df_country[(df_country['Country'] == country) & (df_country['Case'] == case)])\
            for country in peer_countries],
           peer_countries,
           Dark2[n]):
        
        p.line(x='Date', y='Count', line_width=2, color=color, alpha=0.8, legend_label=country, source=data)
        hover = HoverTool(tooltips = [('Country', '@Country'), ('Date', '@DateString'), ('Count','@Count')])
        p.add_tools(hover)
        
    p.legend.location = 'top_left'
    p.legend.click_policy = 'hide'
    
    plots.append(Panel(child=p, title=case))
    
peer_five = Tabs(tabs=plots)
show(peer_five)

## Dashboard

In [ ]:
output_file('../output/bokeh_dashboard.html')

tab1 = Panel(child=world_maps, title='Maps')
tab2 = Panel(child=top_five, title='Top 5')
tab3 = Panel(child=peer_five, title='Nederlands compared to ...')

tabs = Tabs(tabs=[tab1, tab2, tab3])

show(tabs)